In [1]:
from tqdm.notebook import tqdm
import codecs
import pandas as pd
tqdm.pandas()
import string
from multiprocessing import  Pool
import numpy as np
import random

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive/')
# os.chdir("./drive/MyDrive/NLP Project")

Mounted at /content/drive/


In [3]:
# os.listdir()

In [4]:
path="./drive/MyDrive/NLP Project/splitData"


In [5]:
os.listdir(path)

['censored.csv',
 'not_censored.csv',
 'estim.pickle',
 'embeding_data.pickle',
 'autokeras.pickle']

In [6]:
df_data=pd.read_pickle(path+"/embeding_data.pickle")

In [7]:
path

'./drive/MyDrive/NLP Project/splitData'

In [8]:
!pip install autokeras -q

import numpy as np
import pandas as pd
import tensorflow as tf

import autokeras as ak

     |████████████████████████████████| 160 kB 35.1 MB/s 
     |████████████████████████████████| 133 kB 83.1 MB/s 
     |████████████████████████████████| 462 kB 91.5 MB/s 


In [9]:

X=df_data["guse"].to_numpy()
X=np.vstack(X)
y=df_data["permission_denied"].values

test_size = int( 0.2 * len( y ) )
# np.random.seed( seed )
# np.random.RandomState(2)
indices = np.random.permutation(len(X))
x_train = X[ indices[:-test_size]]
y_train = y[ indices[:-test_size]]
x_test = X[ indices[-test_size:]]
y_test = y[ indices[-test_size:]]


In [10]:
train_set = tf.data.Dataset.from_tensor_slices((x_train.astype(np.unicode), y_train))
test_set = tf.data.Dataset.from_tensor_slices(
    (x_test.astype(np.unicode), y_test)
)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.unicode` is a deprecated alias for `np.compat.unicode`. To silence this warning, use `np.compat.unicode` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `str` for which `np.compat.unicode` is itself an alias. Doing this will not modify any behaviour and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.unicode` is a deprecated alias for `np.compat.unicode`. To silence this warning, use `np.compat.unicode` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `str` for which `np.compat.unicode` is itself an alias. Doing th

In [ ]:

clf = ak.StructuredDataClassifier(overwrite=True, max_trials=5)
# Feed the tensorflow Dataset to the classifier.
clf.fit(train_set, epochs=20)
# Predict with the best model.
predicted_y = clf.predict(test_set)
# Evaluate the best model with testing data.
print(clf.evaluate(test_set))

Trial 1 Complete [00h 19m 50s]
val_accuracy: 0.849205493927002

Best val_accuracy So Far: 0.849205493927002
Total elapsed time: 00h 19m 50s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
True              |True              |structured_data_block_1/normalize
False             |False             |structured_data_block_1/dense_block_1/use_batchnorm
2                 |2                 |structured_data_block_1/dense_block_1/num_layers
32                |32                |structured_data_block_1/dense_block_1/units_0
0                 |0                 |structured_data_block_1/dense_block_1/dropout
32                |32                |structured_data_block_1/dense_block_1/units_1
0                 |0                 |classification_head_1/dropout
adam              |adam              |optimizer
0.001             |0.001             |learning_rate

Epoch 1/20
2336/2336 [==============================] - 98s 40ms/step - loss: 0.3719 - accuracy: 0.8347 - val_

In [ ]:
import pickle
with open(path+'/autokeras.pickle', 'wb') as handle:
    pickle.dump(clf, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
clf